In [2]:
from datetime import datetime as dtime
import datetime
import time
import pymysql
import torch
import transformers
from transformers import BartTokenizer, BartForConditionalGeneration
from nltk import sent_tokenize
from summarizer import Summarizer
import warnings
warnings.filterwarnings('ignore')

In [4]:
# 로컬 db 
db = pymysql.connect(host='127.0.0.1', port=3306, user='root',
                         password="1234", db='jisoo', charset='utf8')

In [29]:
#ticker = 'TQQQ' # QQQ, SPY
#checking_sql = "SELECT * FROM US_Stock_Market_News where category = 'etf_news' and symbol = '"+ticker+"' and w_date between date('2022-11-15') and date('2022-11-16') "
checking_sql = "SELECT * FROM US_Stock_Market_News where category = 'etf_news'"
cur = db.cursor()
cur.execute(checking_sql)
result = cur.fetchall()

In [30]:
len(result)

179

In [31]:
ticker_sample = ['QQQ', 'SPY', 'XLE']

In [52]:
news_list = []

news_dict = {'QQQ':[], 'SPY':[], 'XLE':[]}

for r in range(len(result)):
    ticker = result[r][9]
    if ticker not in ticker_sample:
        continue
    data = [result[r][3], result[r][4], result[r][6]]
    news_dict[ticker].append(data)

In [54]:
select_date = '2022-11-15'

data_dict ={'QQQ':[], 'SPY':[], 'XLE':[]}

for i in range(len(ticker_sample)):
    symbols = ticker_sample[i]
    for j in range(len(news_dict[symbols])):
        in_news = news_dict[symbols][j]
        date_str = in_news[1].strftime('%Y-%m-%d')
        if select_date != date_str:
            continue
        data_dict[symbols].append(in_news[0])

In [55]:
len(data_dict['QQQ']), len(data_dict['SPY']), len(data_dict['XLE'])

(7, 6, 4)

In [56]:
data_dict['QQQ'][0]

'When 2022 draws to a close, it’s likely that inflation will be resting at its highest levels since the early 1980s, explaining why growth stocks faltered this year. It remains to be seen, but the October reading of the Consumer Price Index (CPI) declined to 7.7%, which could be a sign that cost pressures are finally abating. If the CPI continues declining in material fashion, exchange traded funds such as the Invesco QQQ Trust (QQQ) and the Invesco NASDAQ 100 ETF (QQQM) could rally. Proving their sensitivity to inflation headlines, QQQ and QQQM surged nearly 9% last week, including 1.84% gains on Friday. That helped the two Invesco ETFs close at their highest levels since September. The duo is higher by 7.12% over the past month. QQQ and QQQM could be set up for better things next year and beyond, assuming inflation cools. “We forecast inflation to average 2.6% over 2022-26 as a whole (in terms of the personal consumption expenditures price index), only slightly above the Fed’s 2% tar

In [65]:
ticker_bell = data_dict['XLE']
doc = []

for p in range(len(ticker_bell)):
    text = ticker_bell[p]
    if len(doc) == 0:
        doc = text  # 본문 
    else:
        doc = doc + ' ' + text

In [66]:
bert_summary = Summarizer()
bsum = bert_summary(doc,  ratio=0.2, num_sentences=3)  # ratio=0.2
bart = bart_generate(bsum)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


In [67]:
bsum

"Designed to provide broad exposure to the Energy - Broad segment of the equity market, the iShares U.S. Energy ETF (IYE) is a passively managed exchange traded fund launched on 06/12/2000. One ETF closely tracking Services stocks is the iShares U.S. Consumer Services ETF (IYC), which is up 1.0% in midday trading, and down 25.80% on a year-to-date basis. Quarterly rebalancing occurs to ensure an individual security doesn't make up more than 25% of the sector."

In [68]:
bart

"The iShares U.S. Consumer Services ETF (IYC) is up 1.0% in midday trading, and down 25.80% on a year-to-date basis. quarterly rebalancing occurs to ensure an individual security doesn't make up more than 25% of the sector."

In [3]:
def bart_summarize(text, num_beams, length_penalty, max_length, min_length, no_repeat_ngram_size):
    model = BartForConditionalGeneration.from_pretrained('facebook/bart-large-cnn')
    tokenizer = BartTokenizer.from_pretrained('facebook/bart-large-cnn')

    torch_device = 'cpu'

    text = text.replace('\n', '')
    text_input_ids = tokenizer.batch_encode_plus([text], return_tensors='pt', max_length=1024)['input_ids'].to(
        torch_device)
    summary_ids = model.generate(text_input_ids, num_beams=int(num_beams), length_penalty=float(length_penalty),
                                 max_length=int(max_length), min_length=int(min_length),
                                 no_repeat_ngram_size=int(no_repeat_ngram_size))
    summary_txt = tokenizer.decode(summary_ids.squeeze(), skip_special_tokens=True)
    return summary_txt


def bart_generate(doc):
    #target = sent_tokenize(doc)
    #target_sent = target[0] + ' ' + target[1] + ' ' + target[2]
    #print(len(target),len(target_sent ))
    target_sent = doc

    ## bart
    num_beams = 4
    length_penalty = 2.0
    max_length =  142
    min_length = 56
    no_repeat_ngram_size = 3
    # 문장별 입력... 별로 좋은 결과는 x
    #bart_1 = bart_summarize(target[0] , num_beams, length_penalty, max_length, min_length, no_repeat_ngram_size)
    #bart_2 = bart_summarize(target[1], num_beams, length_penalty, max_length, min_length, no_repeat_ngram_size)
    #bart_3 = bart_summarize(target[1], num_beams, length_penalty, max_length, min_length, no_repeat_ngram_size)
    #bart = bart_1 + ' ' + bart_2 + ' ' + bart_3

    bart = bart_summarize(target_sent , num_beams, length_penalty, max_length, min_length, no_repeat_ngram_size)

    return bart


## 현재 - 내일 사이의 뉴스 요약
def today_tomorrow(now):
    #today = now.strftime('%Y-%m-%d')
    us_today = now - datetime.timedelta(days=1)  # 미국 시각과 한국 시간 차이
    today = us_today.strftime('%Y-%m-%d')
    now_after = us_today + datetime.timedelta(days=1)
    tomorrow = now_after.strftime('%Y-%m-%d')

    return today, tomorrow

## where press != 'Kiplinger' and


def summary_run(db, today, tomorrow, category):
    #now = dtime.now()
    #today, tomorrow = today_tomorrow(now)

    #today = '2022-07-22'  # 차후 특정 시간대 수집한 뉴스 3가지만 가능하도록...
    #tomorrow = '2022-07-23'  # 차후 특정 시간대 수집한 뉴스 3가지만 가능하도록...

    #print(today, tomorrow)

    title, texts, date = search_sql_news(db,  today, tomorrow, category)
    bert_summary = Summarizer()
    bsum = bert_summary(texts,  ratio=0.2, num_sentences=3)  # ratio=0.2
    bart = bart_generate(bsum)
    subject = bart_generate(title)
    #  이 2개의 원문을 db tabel에 저장할수 있도록
    return subject, bsum, bart, date
